In [98]:
# Import dependencies
from bs4 import BeautifulSoup
import requests
from urllib.parse import quote
import pandas as pd


In [99]:
class GoogleSpider(object):
    def __init__(self):
        """Crawl Google search results

        This class is used to crawl Google's search results using requests and BeautifulSoup.
        """
        super().__init__()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:79.0) Gecko/20100101 Firefox/79.0',
#             'Host': 'www.medium.com',
#             'Referer': 'https://www.medium.com/'
        }

    def __get_source(self, url: str) -> requests.Response:
        """Get the web page's source code

        Args:
            url (str): The URL to crawl

        Returns:
            requests.Response: The response from URL
        """
        return requests.get(url, allow_redirects=True)

    def scrap(self, urls: []) -> list:
        """Search Google

        Args:
            query (str): The query to search for

        Returns:
            list: The search results
        """
        
        df_result = pd.DataFrame(columns = [
                'url',
                'title', 
                'description', 
                'image', 
                'published_date',
                'author',
                'section',
                'type',
              ])
        
        for url in urls:

            # Get response
            response = self.__get_source(url)
            # Initialize BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            # Get the result containers
            result_containers = soup.findAll('head')
            # Final results list
            results = []
            # Loop through every container
            for container in result_containers:

                if container.find('og:title'):
                    title = container.find('og:title').text
                elif container.find('title'):
                    title = container.find('title').text
                else:
                    title = ''

                if container.find('meta', {"property":"og:description"}):
                    description = container.find('meta', {"property":"og:description"})['content']
                else:
                    description = ''

                if container.find('meta', {"property":"og:image"}):
                    image = container.find('meta', {"property":"og:image"})['content']
                else:
                    image = ''

    #             if container.find('meta', {"property":"og:author"}):
    #                 author = container.find('meta', {"property":"og:author"})['content']
    #             else:
    #                 author = ''
    
                if container.find('meta', {"property":"article:published_time"}):
                    published_time = container.find('meta', {"property":"article:published_time"})['content']
                else:
                    published_time = ''

                if container.find('meta', {"property":"article:author"}):
                    article_author = container.find('meta', {"property":"article:author"})['content']
                else:
                    article_author = ''

                if container.find('meta', {"property":"article:section"}):
                    section = container.find('meta', {"property":"article:section"})['content']
                else:
                    section = ''

                if container.find('meta', {"property":"og:type"}):
                    _type = container.find('meta', {"property":"og:type"})['content']
                else:
                    _type = ''

                new_result = {
                    'url': url,
                    'title': title,
                    'description': description,
                    'image': image,
    #                 'author': author,
                    'published_date': published_time,
                    'author': article_author,
                    'section': section,
                    'type': _type,
                }

                df_result = df_result.append(new_result, ignore_index=True)

        return df_result

In [100]:
urls = []
df_url = pd.read_excel('exports/urls.xlsx', index_col=0).reset_index()

for index in df_url.index:
    urls.append(df_url['url'][index])

In [101]:
if __name__ == '__main__':
    df = GoogleSpider().scrap(urls)

In [102]:
df['published_date'] = pd.to_datetime(df['published_date'], format='%Y-%m-%d %H:%M:%S', utc=True).dt.strftime('%d/%m/%Y')

In [103]:
df

,url,title,description,image,published_date,author,section,type
0,https://pasona.co.th/detail_news/92,"News | Job Search, Recruitment Support, Traini...",,,NaN,,,
1,https://pasona.co.th/detail_news/50,"News | Job Search, Recruitment Support, Traini...",,,NaN,,,
2,https://pasona.co.th/detail_news/49,"News | Job Search, Recruitment Support, Traini...",,,NaN,,,
3,https://smartjob.doe.go.th//INF/Post/detail/11...,,,,NaN,,,
4,https://smartjob.doe.go.th//INF/Post/detail/52...,,,,NaN,,,
5,https://blog.jobthai.com/career-tips/5-%E0%B8%...,5 ขั้นตอนปรับ Mindset สำหรับคนทำงานที่อยากพัฒน...,คำว่า Mindset เป็นคำศัพท์ที่นำมาใช้กันบ่อยขึ้น...,https://blog.jobthai.com/service/image/content...,16/11/2020,JobThai,,article
6,https://www.jobtopgun.com/article/%e0%b8%81%e0...,กิจกรรมเพิ่มโอกาสได้งานสำหรับนักศึกษาที่กำลังม...,หางานที่ดีที่สุด @ JOBTOPGUN.COM,//www.jobtopgun.com/article/wp-content/uploads...,28/01/2021,,,article
7,https://www.moneyguru.co.th/lifestyle/articles...,เทคนิคการหางาน หลังวิกฤติการณ์โควิด - MoneyGur...,เทคนิคการหางาน หลังโควิดคลี่คลาย หลายบริษัทอาจ...,https://assets.moneyguru.co.th/blog/th_TH/wp-c...,11/08/2020,,,article
8,https://sitthinunt.com/talent-management/how-t...,5 ขั้นตอนการหางานให้ได้งานสำหรับนักศึกษาจบใหม่...,หลายๆ ครั้งที่ผมรับสมัครคนเข้าทำงาน ผมเห็นผู้ส...,https://sitthinunt.com/wp-content/uploads/2019...,14/01/2019,https://www.facebook.com/bankac120,Talent Management,article
9,https://th.jobsdb.com/th-th/articles/%E0%B9%80...,6 สิ่งที่นายจ้างมองหาจากเด็กจบใหม่ ในยุคโควิด-...,สำหรับนักศึกษาจบใหม่ที่ต้องการหางานและทำยังไงถ...,https://mk0jobsdbth2vucqkbt6.kinstacdn.com/wp-...,27/09/2020,,บทความนักศึกษาจบใหม่,article


In [104]:
df.to_excel('exports/url-specific.xlsx', index=False)